# Mortgage CrossValidation with GPU accelerating on XGBoost

In this notebook, we will show you how to levarage GPU to accelerate mortgage CrossValidation of XGBoost to find out the best model given a group of parameters.

## Import classes
First we need load some common classes that both GPU version and CPU version will use:

In [3]:
import ml.dmlc.xgboost4j.scala.spark.{XGBoostClassificationModel, XGBoostClassifier}

import org.apache.spark.sql.SparkSession
import org.apache.spark.ml.evaluation.MulticlassClassificationEvaluator
import org.apache.spark.ml.tuning.{ParamGridBuilder,CrossValidator}
import org.apache.spark.sql.types.{FloatType, IntegerType, StructField, StructType, DoubleType}

what is new to xgboost-spark users is **rapids.CrossValidator**

## Set dataset path

In [ ]:
// You need to update them to your real paths!
val dataRoot = sys.env.getOrElse("DATA_ROOT", "/data")
val trainParquetPath=dataRoot + "/mortgage/train"
val evalParquetPath=dataRoot + "/mortgage/eval"

# Set the schema of the dataset

In [ ]:
val labelColName = "delinquency_12"
val schema = StructType(List(
    StructField("orig_channel", FloatType),
    StructField("first_home_buyer", FloatType),
    StructField("loan_purpose", FloatType),
    StructField("property_type", FloatType),
    StructField("occupancy_status", FloatType),
    StructField("property_state", FloatType),
    StructField("product_type", FloatType),
    StructField("relocation_mortgage_indicator", FloatType),
    StructField("seller_name", FloatType),
    StructField("mod_flag", FloatType),
    StructField("orig_interest_rate", FloatType),
    StructField("orig_upb", DoubleType),
    StructField("orig_loan_term", IntegerType),
    StructField("orig_ltv", FloatType),
    StructField("orig_cltv", FloatType),
    StructField("num_borrowers", FloatType),
    StructField("dti", FloatType),
    StructField("borrower_credit_score", FloatType),
    StructField("num_units", IntegerType),
    StructField("zip", IntegerType),
    StructField("mortgage_insurance_percent", FloatType),
    StructField("current_loan_delinquency_status", IntegerType),
    StructField("current_actual_upb", FloatType),
    StructField("interest_rate", FloatType),
    StructField("loan_age", FloatType),
    StructField("msa", FloatType),
    StructField("non_interest_bearing_upb", FloatType),
    StructField(labelColName, IntegerType)))

## Create a new spark session and load data
we must create a new spark session to continue all spark operations.

NOTE: in this notebook, we have uploaded dependency jars when installing toree kernel. If we don't upload them at installation time, we can also upload in notebook by [%AddJar magic](https://toree.incubator.apache.org/docs/current/user/faq/). However, there's one restriction for `%AddJar`: the jar uploaded can only be available when `AddJar` is called after a new spark session is created. We must use it as below:

```scala
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().appName("mortgage-gpu-cv").getOrCreate
%AddJar file:/data/libs/xgboost4j-spark-gpu_2.12-XXX.jar
%AddJar file:/data/libs/xgboost4j-gpu_2.12-XXX.jar
// ...
```

In [8]:
val spark = SparkSession.builder().appName("mortgage-gpu-cv").getOrCreate()
val trainDs = spark.read.parquet(trainParquetPath)

spark = org.apache.spark.sql.SparkSession@51af6ff3
trainDs = [orig_channel: double, first_home_buyer: double ... 26 more fields]


[orig_channel: double, first_home_buyer: double ... 26 more fields]

## Find out features to train

In [9]:
val featureNames = schema.filter(_.name != labelColName).map(_.name).toArray

featureNames = Array(orig_channel, first_home_buyer, loan_purpose, property_type, occupancy_status, property_state, product_type, relocation_mortgage_indicator, seller_name, mod_flag, orig_interest_rate, orig_upb, orig_loan_term, orig_ltv, orig_cltv, num_borrowers, dti, borrower_credit_score, num_units, zip, mortgage_insurance_percent, current_loan_delinquency_status, current_actual_upb, interest_rate, loan_age, msa, non_interest_bearing_upb)


Array(orig_channel, first_home_buyer, loan_purpose, property_type, occupancy_status, property_state, product_type, relocation_mortgage_indicator, seller_name, mod_flag, orig_interest_rate, orig_upb, orig_loan_term, orig_ltv, orig_cltv, num_borrowers, dti, borrower_credit_score, num_units, zip, mortgage_insurance_percent, current_loan_delinquency_status, current_actual_upb, interest_rate, loan_age, msa, non_interest_bearing_upb)

In [13]:
val classifierParam = Map(
    "objective" -> "binary:logistic",
    "num_round" -> 100,
    "num_workers" -> 1,
    "tree_method" -> "gpu_hist")

classifierParam = Map(objective -> binary:logistic, num_round -> 100, num_workers -> 1, tree_method -> gpu_hist)


Map(objective -> binary:logistic, num_round -> 100, num_workers -> 1, tree_method -> gpu_hist)

## Construct CrossValidator

In [14]:
val classifier = new XGBoostClassifier(classifierParam)
    .setLabelCol(labelColName)
    .setFeaturesCol(featureNames)
val paramGrid = new ParamGridBuilder()
    .addGrid(classifier.maxDepth, Array(3, 10))
    .addGrid(classifier.eta, Array(0.2, 0.6))
    .build()
val evaluator = new MulticlassClassificationEvaluator().setLabelCol(labelColName)
val cv = new CrossValidator()
    .setEstimator(classifier)
    .setEvaluator(evaluator)
    .setEstimatorParamMaps(paramGrid)
    .setNumFolds(3)

classifier = xgbc_ae8896ab2b67
paramGrid = 
evaluator = MulticlassClassificationEvaluator: uid=mcEval_ebda5b6cea6c, metricName=f1, metricLabel=0.0, beta=1.0, eps=1.0E-15
cv = cv_cb7d8efe9ab5


Array({
	xgbc_ae8896ab2b67-eta: 0.2,
	xgbc_ae8896ab2b67-maxDepth: 3
}, {
	xgbc_ae8896ab2b67-eta: 0.2,
	xgbc_ae8896ab2b67-maxDepth: 10
}, {
	xgbc_ae8896ab2b67-eta: 0.6,
	xgbc_ae8896ab2b67-maxDepth: 3
}, {
	xgbc_ae8896ab2b67-eta: 0.6,
	xgbc_ae8896ab2b67-maxDepth: 10
})


cv_cb7d8efe9ab5

## train with CrossValidator

In [15]:
val model = cv.fit(trainDs).bestModel.asInstanceOf[XGBoostClassificationModel]

Tracker started, with env={DMLC_NUM_SERVER=0, DMLC_TRACKER_URI=10.19.183.210, DMLC_TRACKER_PORT=41609, DMLC_NUM_WORKER=1}
Tracker started, with env={DMLC_NUM_SERVER=0, DMLC_TRACKER_URI=10.19.183.210, DMLC_TRACKER_PORT=45469, DMLC_NUM_WORKER=1}
Tracker started, with env={DMLC_NUM_SERVER=0, DMLC_TRACKER_URI=10.19.183.210, DMLC_TRACKER_PORT=52795, DMLC_NUM_WORKER=1}
Tracker started, with env={DMLC_NUM_SERVER=0, DMLC_TRACKER_URI=10.19.183.210, DMLC_TRACKER_PORT=53483, DMLC_NUM_WORKER=1}
Tracker started, with env={DMLC_NUM_SERVER=0, DMLC_TRACKER_URI=10.19.183.210, DMLC_TRACKER_PORT=58067, DMLC_NUM_WORKER=1}
Tracker started, with env={DMLC_NUM_SERVER=0, DMLC_TRACKER_URI=10.19.183.210, DMLC_TRACKER_PORT=43717, DMLC_NUM_WORKER=1}
Tracker started, with env={DMLC_NUM_SERVER=0, DMLC_TRACKER_URI=10.19.183.210, DMLC_TRACKER_PORT=36075, DMLC_NUM_WORKER=1}
Tracker started, with env={DMLC_NUM_SERVER=0, DMLC_TRACKER_URI=10.19.183.210, DMLC_TRACKER_PORT=53851, DMLC_NUM_WORKER=1}
Tracker started, with en

model = xgbc_ae8896ab2b67


xgbc_ae8896ab2b67

## tranform with best model trained by CrossValidator

In [16]:
val transformDs = spark.read.parquet(evalParquetPath)
val df = model.transform(transformDs).cache()
df.drop(featureNames: _*).show(5)

transformDs = [orig_channel: double, first_home_buyer: double ... 26 more fields]
df = [orig_channel: double, first_home_buyer: double ... 29 more fields]


+--------------+--------------------+--------------------+----------+
|delinquency_12|       rawPrediction|         probability|prediction|
+--------------+--------------------+--------------------+----------+
|             0|[17.3849449157714...|[0.99999997182821...|       0.0|
|             0|[16.6074829101562...|[0.99999993869981...|       0.0|
|             0|[16.0062618255615...|[0.99999988816731...|       0.0|
|             0|[16.7623615264892...|[0.99999994749521...|       0.0|
|             0|[15.1363153457641...|[0.99999973307967...|       0.0|
+--------------+--------------------+--------------------+----------+
only showing top 5 rows



[orig_channel: double, first_home_buyer: double ... 29 more fields]

In [17]:
val evaluator = new MulticlassClassificationEvaluator().setLabelCol(labelColName)
val accuracy = evaluator.evaluate(df)

evaluator = MulticlassClassificationEvaluator: uid=mcEval_d880c25944f1, metricName=f1, metricLabel=0.0, beta=1.0, eps=1.0E-15
accuracy = 1.0


1.0

In [12]:
spark.close()